# thre fist part is versification of Pytorch FCN & ResNet implementation

In [1]:
import os
import sys
currentdir = os.path.dirname(os.path.abspath("__file__"))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

import torch
import torch.nn as nn
from os.path import dirname
import numpy as np
from utils.log_manager import eval_condition, eval_model
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from utils.dataloader.TSC_data_loader import TSC_data_loader
from Classifiers.FCN import FCN as Torch_FCN

Max_epoch = 2000
print_result_every_x_epoch = 200


In [2]:
dataset_name_list = [
    'FiftyWords',  # 62.7(6.1)
]
# check our result with result here 
# https://github.com/hfawaz/dl-4-tsc/blob/master/results/results-uea-avg-std.csv

dataset_path = dirname(parentdir+"./Example_Datasets/UCRArchive_2018/")
for dataset_name in dataset_name_list:
    print(dataset_name)
    X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('code is running on: ', device)
    X_train = torch.from_numpy(X_train)
    X_train.requires_grad = False
    X_train = X_train.unsqueeze_(1).to(device)
    y_train = torch.from_numpy(y_train).to(device)
    X_test = torch.from_numpy(X_test)
    X_test.requires_grad = False
    X_test = X_test.unsqueeze_(1).to(device)
    y_test = torch.from_numpy(y_test).to(device)

    input_shape = X_train.shape[-1]
    n_class = max(y_train) + 1
    print(input_shape, n_class)
    
    
    torch_FCN = Torch_FCN(input_shape, n_class.item()).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(torch_FCN.parameters())
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience= 50, min_lr=0.0001)
    torch_FCN.train()

    dataset = TensorDataset(X_train, y_train)
    batch_size = 16
    loader = DataLoader(dataset, batch_size=int(min(X_train.shape[0] / 10, batch_size)), shuffle=True)

    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=int(min(X_train.shape[0] / 10, batch_size)), shuffle=False)

    for i in range(Max_epoch):
        for sample in loader:
            optimizer.zero_grad()
            y_predict = torch_FCN(sample[0])
            output = criterion(y_predict, sample[1])
            output.backward()
            optimizer.step()
        scheduler.step(output)

        if eval_condition(i,print_result_every_x_epoch):
            for param_group in optimizer.param_groups:
                print(i, output.item(), param_group['lr'])
            torch_FCN.eval()
            acc_train = eval_model(torch_FCN, loader)
            acc_test = eval_model(torch_FCN, test_loader)
            torch_FCN.train()
            print('train_acc=\t', acc_train, '\t test_acc=\t', acc_test,'\t loss=\t', output.item())

    torch_FCN.eval()
    acc_test = eval_model(torch_FCN, test_loader)
    torch.cuda.empty_cache()
    print(dataset_name,acc_test)


FiftyWords
code is running on:  cuda:0
270 tensor(50, device='cuda:0')
199 4.171777725219727 0.00025
train_acc=	 0.9555555555555556 	 test_acc=	 0.643956043956044 	 loss=	 4.171777725219727
399 0.1811048984527588 0.000125
train_acc=	 0.9977777777777778 	 test_acc=	 0.6461538461538462 	 loss=	 0.1811048984527588
599 1.843912959098816 0.0001
train_acc=	 0.9844444444444445 	 test_acc=	 0.6043956043956044 	 loss=	 1.843912959098816
799 0.3334866166114807 0.0001
train_acc=	 1.0 	 test_acc=	 0.6395604395604395 	 loss=	 0.3334866166114807
999 0.35695481300354004 0.0001
train_acc=	 1.0 	 test_acc=	 0.6681318681318681 	 loss=	 0.35695481300354004
1199 2.666208505630493 0.0001
train_acc=	 1.0 	 test_acc=	 0.6681318681318681 	 loss=	 2.666208505630493
1399 0.2539825439453125 0.0001
train_acc=	 1.0 	 test_acc=	 0.6351648351648351 	 loss=	 0.2539825439453125
1599 0.6584628820419312 0.0001
train_acc=	 1.0 	 test_acc=	 0.6461538461538462 	 loss=	 0.6584628820419312
1799 0.387653112411499 0.0001
train

In [3]:
from Classifiers.ResNet import ResNet as Torch_ResNet

print_result_every_x_epoch = 200

dataset_name_list = [
    'FiftyWords',  # 74.0(1.5)
]
# check our result with result here 
# https://github.com/hfawaz/dl-4-tsc/blob/master/results/results-uea-avg-std.csv

dataset_path = dirname(parentdir+"./Example_Datasets/UCRArchive_2018/")
for dataset_name in dataset_name_list:
    print(dataset_name)
    X_train, y_train, X_test, y_test = TSC_data_loader(dataset_path, dataset_name)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print('code is running on: ', device)
    X_train = torch.from_numpy(X_train)
    X_train.requires_grad = False
    X_train = X_train.unsqueeze_(1).to(device)
    y_train = torch.from_numpy(y_train).to(device)
    X_test = torch.from_numpy(X_test)
    X_test.requires_grad = False
    X_test = X_test.unsqueeze_(1).to(device)
    y_test = torch.from_numpy(y_test).to(device)

    input_shape = X_train.shape[-1]
    n_class = max(y_train) + 1
    print(input_shape, n_class)
    
    
    Torch_ResNet = Torch_ResNet(input_shape, n_class.item()).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(Torch_ResNet.parameters())
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience= 50, min_lr=0.0001)
    Torch_ResNet.train()

    dataset = TensorDataset(X_train, y_train)
    batch_size = 16
    loader = DataLoader(dataset, batch_size=int(min(X_train.shape[0] / 10, batch_size)), shuffle=True)

    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=int(min(X_train.shape[0] / 10, batch_size)), shuffle=False)

    for i in range(Max_epoch):
        for sample in loader:
            optimizer.zero_grad()
            y_predict = Torch_ResNet(sample[0])
            output = criterion(y_predict, sample[1])
            output.backward()
            optimizer.step()
        scheduler.step(output)

        if eval_condition(i,print_result_every_x_epoch):
            for param_group in optimizer.param_groups:
                print(i, output.item(), param_group['lr'])
            Torch_ResNet.eval()
            acc_train = eval_model(Torch_ResNet, loader)
            acc_test = eval_model(Torch_ResNet, test_loader)
            Torch_ResNet.train()
            print('train_acc=\t', acc_train, '\t test_acc=\t', acc_test,'\t loss=\t', output.item())

    Torch_ResNet.eval()
    acc_test = eval_model(Torch_ResNet, test_loader)
    torch.cuda.empty_cache()
    print(dataset_name,acc_test)


FiftyWords
code is running on:  cuda:0
270 tensor(50, device='cuda:0')
199 1.6963386535644531 0.001
train_acc=	 1.0 	 test_acc=	 0.7186813186813187 	 loss=	 1.6963386535644531
399 0.299480140209198 0.00025
train_acc=	 1.0 	 test_acc=	 0.734065934065934 	 loss=	 0.299480140209198
599 0.5820592641830444 0.0001
train_acc=	 1.0 	 test_acc=	 0.7208791208791209 	 loss=	 0.5820592641830444
799 0.8479273319244385 0.0001
train_acc=	 1.0 	 test_acc=	 0.7142857142857143 	 loss=	 0.8479273319244385
999 1.4371581077575684 0.0001
train_acc=	 1.0 	 test_acc=	 0.7274725274725274 	 loss=	 1.4371581077575684
1199 0.045929908752441406 0.0001
train_acc=	 1.0 	 test_acc=	 0.7450549450549451 	 loss=	 0.045929908752441406
1399 0.021345853805541992 0.0001
train_acc=	 1.0 	 test_acc=	 0.7274725274725274 	 loss=	 0.021345853805541992
1599 0.05151724815368652 0.0001
train_acc=	 1.0 	 test_acc=	 0.7032967032967034 	 loss=	 0.05151724815368652
1799 0.25116491317749023 0.0001
train_acc=	 1.0 	 test_acc=	 0.72967032